## Automatizando processo de envio de mensagens WhatsApp

* Usar o https://web.whatsapp.com/send?phone=numero&text=texto (mais fácil, mais seguro, mas mais demorado)

In [ ]:
import os
import time
import urllib
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

def inicializar_navegador():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    return webdriver.Chrome(options=options)

def encontrar_elemento_clickavel(driver, metodo, seletor, tempo=20):
    return WebDriverWait(driver, tempo).until(EC.element_to_be_clickable((metodo, seletor)))

def encontrar_elemento(driver, metodo, seletor, tempo=30):
    return WebDriverWait(driver, tempo).until(EC.presence_of_element_located((metodo, seletor)))
    
def enviar_anexo_mensagem(driver, caminho_arquivo):
    try:
        print(f"Anexando arquivo: {caminho_arquivo}")
        encontrar_elemento_clickavel(driver, By.XPATH, '//*[@id="main"]/footer/div[1]/div/span/div/div[1]/div/button/span').click()
        time.sleep(1)

        input_file = driver.find_element(By.CSS_SELECTOR, 'input[type="file"]')
        input_file.send_keys(caminho_arquivo)

        time.sleep(5)

        botao_enviar = encontrar_elemento_clickavel(driver, By.XPATH, '//div[contains(@aria-label, "Enviar")]')
        botao_enviar.click()
        print("✅ Arquivo enviado com sucesso!")

    except Exception as e:
        print(f"❌ Falha ao enviar o anexo: {e}")

def enviar_mensagem(driver, telefone, nome, mensagem, arquivo_anexo):
    texto = urllib.parse.quote(mensagem.replace("fulano", nome))
    link = f"https://web.whatsapp.com/send?phone={telefone}&text={texto}"

    try:
        driver.get(link)
        encontrar_elemento(driver, By.ID, "side")
        time.sleep(3)

        # verificando a veracidade do número (ou seja, se a caixa de erro não aparecer)
        if len(driver.find_elements(By.XPATH, '//*[@id="app"]/div/span[2]/div/span/div/div/div/div/div/div[1]')) < 1:
            encontrar_elemento_clickavel(driver, By.XPATH, '//*[@id="main"]/footer/div[1]/div/span/div/div[2]/div[2]/button/span').click()
            print(f"✅ Mensagem enviada para {nome} => {telefone}\n")

            # caso houver anexo e o mesmo for válido
            if arquivo_anexo != "N":
                caminho_arquivo = os.path.abspath(f"arquivos/{arquivo_anexo}")
                if os.path.exists(caminho_arquivo):
                    enviar_anexo_mensagem(driver, caminho_arquivo)
                else:
                    print(f"⚠️ Arquivo não encontrado: {caminho_arquivo}")
        else:
            print(f"⚠️ Número inválido ou erro ao tentar abrir a conversa para o contato: {telefone}")
    except Exception as e:
        print(f"❌ Erro ao enviar as mensagens para {nome} - {telefone}: {e}")

# ----------- PROGRAMA GERAL -----------

if __name__ == "__main__":
    navegador = inicializar_navegador()
    navegador.get("https://web.whatsapp.com")
    print("🔓 Aguardando o login ser efetuado no WhatsApp Web...")
    encontrar_elemento(navegador, By.ID, "side")
    print("✅ Login realizado com sucesso.. \n")

    tabela_envio_mensagens = pd.read_excel("Envios.xlsx")

    for linha in tqdm(tabela_envio_mensagens.index, desc="📤 Enviando mensagens"):
        nome = tabela_envio_mensagens.loc[linha, "nome"]
        mensagem = tabela_envio_mensagens.loc[linha, "mensagem"]
        arquivo_anexo = tabela_envio_mensagens.loc[linha, "arquivo"]
        telefone = tabela_envio_mensagens.loc[linha, "telefone"]

        enviar_mensagem(navegador, telefone, nome, mensagem, arquivo_anexo)
        time.sleep(5) # pausa para controle entre os envios

    print("\n🏁 Processo de automação de envio de mensagens via WhatsApp concluído com sucesso!")
    # navegador.quit()